In [6]:
import keras
from keras.datasets import cifar10
import numpy as np
from keras.applications.vgg16 import VGG16, preprocess_input
import scipy
from scipy import misc
import os

# load the data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = np.squeeze(y_train)
print('data loaded')

# load VGG16 model + remove final classification layers
I_R=64
model = VGG16(weights='imagenet', include_top=False, input_shape=(I_R,I_R, 3))
model.summary()
print('model loaded')

data loaded
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0        

In [7]:
# obtain bottleneck features (train)
if os.path.exists('vgg16_features_train.npz'):
    print('bottleneck features detected (train)')
    features = np.load('vgg16_features_train.npz')['features']
else:
  # obtain bottleneck features (train)
  print('bottleneck features file not detected (train)')
  print('calculating now ...')
  # pre-process the train data
  big_x_train = np.array([scipy.misc.imresize(x_train[i], (I_R,I_R, 3)) 
                     for i in range(0, len(x_train))]).astype('float32')
  inception_input_train = preprocess_input(big_x_train)
  print('train data prepsrocessed')
  # extract, process, and save bottleneck features
  features = model.predict(inception_input_train)
  features = np.squeeze(features)
  np.savez('vgg16_features_train', features=features)
print('bottleneck features saved (train)')

bottleneck features detected (train)
bottleneck features saved (train)


In [8]:
# obtain bottleneck features (test)
if os.path.exists('vgg16_features_test.npz'):
    print('bottleneck features detected (test)')
    features_test = np.load('vgg16_features_test.npz')['features_test']
else:
  print('bottleneck features file not detected (test)')
  print('calculating now ...')
  # pre-process the test data
  big_x_test = np.array([scipy.misc.imresize(x_test[i], (I_R,I_R, 3)) 
                     for i in range(0, len(x_test))]).astype('float32')
  inception_input_test = preprocess_input(big_x_test)
  # extract, process, and save bottleneck features (test)
  features_test = model.predict(inception_input_test)
  features_test = np.squeeze(features_test)
  np.savez('vgg16_features_test', features_test=features_test)
print('bottleneck features saved (test)')

bottleneck features detected (test)
bottleneck features saved (test)


In [0]:
from keras.utils import np_utils

# one-hot encode the labels
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

In [10]:
from keras.callbacks import ModelCheckpoint   
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, GlobalAveragePooling2D

model = Sequential()
model.add(Conv2D(filters=100, kernel_size=2, input_shape=features.shape[1:]))
model.add(Dropout(0.4))
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.3))
model.add(Dense(10, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath='cnn_vgg16_model.best.hdf5', 
                               verbose=1, save_best_only=True)
model.fit(features, y_train, batch_size=50, epochs=50,
          validation_split=0.2, callbacks=[checkpointer],
          verbose=2, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 1, 1, 100)         204900    
_________________________________________________________________
dropout_3 (Dropout)          (None, 1, 1, 100)         0         
_________________________________________________________________
global_average_pooling2d_2 ( (None, 100)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 205,910
Trainable params: 205,910
Non-trainable params: 0
_________________________________________________________________
Train on 40000 samples, validate on 10000 samples
Epoch 1/50
 - 5s - loss: 5.8290 - acc: 0.5961 - val_loss: 4.0609 - val_acc: 0.7154

Ep

 - 5s - loss: 3.6330 - acc: 0.7642 - val_loss: 3.6011 - val_acc: 0.7668

Epoch 00020: val_loss did not improve
Epoch 21/50
 - 5s - loss: 3.6309 - acc: 0.7643 - val_loss: 3.4294 - val_acc: 0.7769

Epoch 00021: val_loss improved from 3.48081 to 3.42944, saving model to cnn_vgg16_model.best.hdf5
Epoch 22/50
 - 5s - loss: 3.5851 - acc: 0.7670 - val_loss: 3.4470 - val_acc: 0.7752

Epoch 00022: val_loss did not improve
Epoch 23/50
 - 5s - loss: 3.5829 - acc: 0.7681 - val_loss: 3.4603 - val_acc: 0.7744

Epoch 00023: val_loss did not improve
Epoch 24/50
 - 5s - loss: 3.6138 - acc: 0.7664 - val_loss: 3.5247 - val_acc: 0.7720

Epoch 00024: val_loss did not improve
Epoch 25/50
 - 5s - loss: 3.6038 - acc: 0.7664 - val_loss: 3.5159 - val_acc: 0.7710

Epoch 00025: val_loss did not improve
Epoch 26/50
 - 5s - loss: 3.6050 - acc: 0.7677 - val_loss: 3.5034 - val_acc: 0.7733

Epoch 00026: val_loss did not improve
Epoch 27/50
 - 5s - loss: 3.5988 - acc: 0.7677 - val_loss: 3.5196 - val_acc: 0.7719

Epoch 


Epoch 00047: val_loss did not improve
Epoch 48/50
 - 5s - loss: 3.4965 - acc: 0.7760 - val_loss: 3.5686 - val_acc: 0.7725

Epoch 00048: val_loss did not improve
Epoch 49/50
 - 5s - loss: 3.5305 - acc: 0.7743 - val_loss: 3.5218 - val_acc: 0.7753

Epoch 00049: val_loss did not improve
Epoch 50/50
 - 5s - loss: 3.4454 - acc: 0.7800 - val_loss: 3.5344 - val_acc: 0.7743

Epoch 00050: val_loss did not improve


In [11]:
# load the weights that yielded the best validation accuracy
model.load_weights('cnn_vgg16_model.best.hdf5')

# evaluate test accuracy
score = model.evaluate(features_test, y_test, verbose=0)
accuracy = 100*score[1]

# print test accuracy
print('Test accuracy: %.4f%%' % accuracy)

Test accuracy: 76.4900%
